In [8]:
import CSV

import MyUPFLOW
up = MyUPFLOW

path_design1 = "design20180903";

# MyTrainer 1.0 &ndash; Design Requirements

**NEEDS**
1. I need something easy to fly so that I can get experience piloting.
2. I need something easy to design so that I can get a feeling of the design process described by Raymer and test my aerodynamics and stability tools.
3. I need something easy to manufacture so that I can get initial experience building model airplanes.
4. There is no need for avionics.

**VISION**
> A very stable and easy take-off trainer airplane that can be used to gain piloting experience of conventional configurations.

**BUILD REQUIREMENTS**
* Conventional configuration aircraft: Wing, vertical and horizontal tail (ailerons, stabilizer, and rudder).
* Inherently (passively) stable.
* Smallest as possible.

**MISSION REQUIREMENTS**
* No more than one second of descent during take off.

Raymer's *Aircraft Design: A Conceptual Approach* outlines the following design process.

<img src="img/IMG_4012.JPG" alt="Pic here" style="width:700px">

For the sake of trying it out, let's stick to it at much as it is reasonable.

# Technology Availability

This is an initial listing of electronic components to be used:

In [7]:
electronics = CSV.read(joinpath(path_design1, "electronics.csv"))
display(electronics)

m_elec = sum(electronics[2].*electronics[3])/1000 # (kg) mass of electronics
println("Total mass of electronics:\t$(m_elec*1000) (g)")

,Item,Quantity,Weight (g),Cost ($),Vendor,Description
1,battery,1,36,missing,missing,missing
2,motor,2,15,missing,missing,missing
3,ESC,2,9,missing,missing,missing
4,BEC,1,16,missing,missing,missing
5,propeller,2,3,missing,missing,missing
6,servos,4,9,missing,missing,missing
7,wires,1,50,missing,missing,missing
8,receiver,1,15,missing,missing,missing


Total mass of electronics:	207.0 (g)


# Concept Sketch

This is somewhat what I envision

<img src="img/IMG_4013.JPG" alt="Pic here" style="width:700px">

# Initial Sizing

The mass of the fuselage ($M_f$) will be assumed to be zero using a plastic water bottle as the fuselage.

The mass of the wing ($M_w$), made out of HD EPS foam of volumetric density $\rho_f$ and carbon fiber spars of linear density $\rho_\text{spar}$, will be approximated as 

\begin{align}
        \bullet \quad & M_w = 0.5 \cdot 2 \rho_f \Big[ 
            \left(\frac{c_{wt}+c_{wr}}{2}\right) \frac{b_w/2}{\cos\Lambda_w} 
            \left(\frac{c_{wt}+c_{wr}}{2}\right) \left(\frac{t_{wt}^* + t_{wr}^*}{2}\right) \Big] +
            \rho_\text{spar}\frac{b_w}{\cos\Lambda_w} 
    \\ & \qquad = 0.5 \cdot \rho_f \frac{b_w}{\cos\Lambda_w} 
            \left(\frac{c_{wt}+c_{wr}}{2}\right)^2
            \left(\frac{t_{wt}^* + t_{wr}^*}{2}\right) +
            \rho_\text{spar}\frac{b_w}{\cos\Lambda_w} 
\end{align}

with $t^*$ the relative airfoil thickness $t^*=t/c$. The same formula is used for the mass of the horizontal stabilizer $M_h$, and the mass of the vertical stabilizer $M_v$ is the formula multiplied by $0.5$. 

Here is my initial guess of the sizing:

In [12]:
include(joinpath(path_design1, "parameters.jl"))

open(joinpath(path_design1, "parameters.jl")) do file
    for ln in eachline(file)
        println("$(ln)")
    end
end

##############################################################################
#   PARAMETERS
##############################################################################
# ------ CONSTANTS
g = 9.82            # (m/s^2) gravity
rhoinf= 1.225       # (kg/m^3) air density at sea level and 15Cdegs
mu = 1.846/10^5     # (kg/m*s) air dynamic viscosity

# ------ DESIGN PARAMETERS
# magVinf = 5.40    # (m/s) optimum cruise speed

# ------ MATERIALS
rho_c = 0.483       # (kg/m^2) Cloroplast area density
rho_f = 24.8        # (kg/m^3) HD EPS foam volumetric density
rho_spar = 50/1000/0.75     # (kg/m) Carbon fiber spar linear density

# ------ GEOMETRIC PARAMETERS
# # Fuselage
# w = 8/100         # (m) width
# l = (560.000-230.000)/1000          # (m) length

# Wing
b_w = 0.75          # (m) span
lambda_w = 0        # (deg) sweep
c_wroot = 0.20      # (m) root chord
t_wroot = 0.10      # (ratio) max thickness/chord at root
c_wtip = 0.10       # (m) tip chord
t_wtip = 0.20       # (ratio) max 

In [24]:

# Mass of wing
M_w = up.mass_wing(rho_f, b_w, c_wroot, c_wtip, t_wroot, t_wtip, lambda_w; 
                                                    rho_spar=rho_spar, f=0.5)
# Mass of horizontal stabilizer
M_h = up.mass_wing(rho_f, b_h, c_hroot, c_htip, t_hroot, t_htip, lambda_h; 
                                                    rho_spar=rho_spar, f=0.5)
# Mass of vertical stabilizer
M_v = 0.5*up.mass_wing(rho_f, 2*h_v, c_vroot, c_vtip, t_vroot, t_vtip, lambda_v; 
                                                    rho_spar=rho_spar, f=0.5)
# Unpredicted additional mass
M_un = 100/1000

M_tot = M_w + M_h + M_v + m_elec + M_un

println("****** MASS PER COMPONENT ******")
println("Wing:\t\t\t$(round(M_w*1000,0)) g")
println("Horizontal stabilizer:\t$(round(M_h*1000,0)) g")
println("Vertical stabilizer:\t$(round(M_v*1000,0)) g")
println("Electronics:\t\t$(round(m_elec*1000,0)) g")
println("Unpredicted components:\t$(round(M_un*1000,0)) g")
println("--------------------------------")
println("TOTAL MASS:\t\t$(round(M_tot*1000,0)) g")
println("--------------------------------")

****** MASS PER COMPONENT ******
Wing:			81.0 g
Horizontal stabilizer:	13.0 g
Vertical stabilizer:	11.0 g
Electronics:		207.0 g
Unpredicted components:	100.0 g
--------------------------------
TOTAL MASS:		412.0 g
--------------------------------


# TODO

* [ ] Visualize current sizing.